In [2]:
# Image only classification Basic

In [1]:
import numpy as np
import pandas as pd
import os
from time import time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_files       
from keras.utils import np_utils
from glob import glob
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from sklearn.model_selection import train_test_split

In [2]:
def read_image(file_path):
    image = load_img(file_path, target_size=(50, 50))
    image = img_to_array(image) 
    image = np.expand_dims(image, axis=0)
    image /= 255. 
    return image

In [3]:
df = pd.read_json('/home/jovyan/data/train.jsonl', lines=True)
df_test = pd.read_json('/home/jovyan/data/dev.jsonl', lines=True)
#df_test = pd.read_json('/home/jovyan/data/test.jsonl', lines=True)
df[df.label == 1].head(2)

,id,img,label,text
10,79351,img/79351.png,1,jew mad? get fuhrerious!
12,25489,img/25489.png,1,brother... a day without a blast is a day wasted


In [4]:
#df["image_data"] = df.apply(lambda x: read_image('/home/jovyan/data/' + x['img']), axis=1)
#df.to_pickle("df.pkl")

#df_test["image_data"] = df_test.apply(lambda x: read_image('/home/jovyan/data/' + x['img']), axis=1)
#df_test.to_pickle("df_test.pkl")

In [5]:
#df.to_csv('df.csv', index=False)
df = pd.read_pickle('df.pkl')

In [6]:
x = np.vstack(df["image_data"])
y = df["label"]
del df

In [7]:
#train_tensors, train_targets, valid_tensors, valid_targets = train_test_split(x, y, test_size=0.1, random_state=123)
train_tensors = x
train_targets = y

In [30]:
df_test = pd.read_pickle('df_test.pkl')
test_tensors = np.vstack(df_test["image_data"])
test_targets = df_test["label"]
del df_test

In [31]:

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()
print('Input image sizes', train_tensors[0].shape)
### TODO: Define your architecture.
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=train_tensors[0].shape ))

model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))

model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))

model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=128, kernel_size=2, strides=2, padding='same', activation='relu'))

model.add(GlobalAveragePooling2D())

model.add(Dropout(0.4))
model.add(Dense(1, activation='softmax'))

model.summary()

Input image sizes (50, 50, 3)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_15 (Conv2D)      

In [32]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC'])

In [35]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 2

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='/home/jovyan/data/saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, validation_data=(test_tensors, test_targets),
          epochs=epochs, batch_size=2, callbacks=[checkpointer], verbose=1)

Epoch 1/2
4248/4250 [============================>.] - ETA: 0s - loss: 9.7788 - accuracy: 0.3588 - auc: 0.5000
Epoch 00001: val_loss improved from inf to 7.62463, saving model to /home/jovyan/data/saved_models/weights.best.from_scratch.hdf5
4250/4250 [==============================] - 28s 7ms/step - loss: 9.7778 - accuracy: 0.3588 - auc: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000 - val_auc: 0.5000
Epoch 2/2
4243/4250 [============================>.] - ETA: 0s - loss: 9.7741 - accuracy: 0.3591 - auc: 0.5000
Epoch 00002: val_loss did not improve from 7.62463
4250/4250 [==============================] - 28s 7ms/step - loss: 9.7777 - accuracy: 0.3588 - auc: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000 - val_auc: 0.5000


In [25]:
# model.load_weights('/home/jovyan/data/saved_models/weights.best.from_scratch.hdf5')


In [36]:
# get index of predicted dog breed for each image in test set
predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(predictions==test_targets)/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 50.0000%


In [40]:
predictions[predictions==1]

0